In [ ]:
import xarray as xr
import os
import numpy as np

NEIGHBORHOOD = 1
OBS_DATA_PATH = "../data/processed/obs/"
RESULT_PATH = "../data/neighbourhood/graphcast_{NEIGHBORHOOD}/"

In [ ]:
obs_file_list = sorted([f for f in os.listdir(OBS_DATA_PATH) if f.endswith(".nc")])
results_file_list = sorted([f for f in os.listdir(RESULT_PATH) if f.endswith(".nc")])

In [ ]:
def get_point_fcst(fcst, longitude, latitude):
    latitude = np.round(latitude / 0.25) * 0.25
    longitude = 360 + np.round(longitude / 0.25) * 0.25
    point_fcst = fcst.sel(latitude=latitude, longitude=longitude, method="nearest")
    return point_fcst

In [ ]:
fcst = xr.open_dataset("../data/processed/graphcast/graphcast.nc")

In [ ]:
# CONUS domain
latN = 50.4
latS = 24.25
lonW = -126
lonE = -66

obs_metadata = []
for obs_file_name in obs_file_list:
    obs = xr.open_dataarray(f"{OBS_DATA_PATH}{obs_file_name}")
    station = obs.attrs["station"]
    latitude = obs.attrs["lat"]
    longitude = obs.attrs["lon"]
    obs_meta_dict = {"station": station, "latitude": latitude, "longitude": longitude}
    obs_metadata.append(obs_meta_dict)


point_list = []
for obs_meta in obs_metadata:
    station = obs_meta["station"]
    latitude = obs_meta["latitude"]
    longitude = obs_meta["longitude"]
    if longitude < lonW or longitude > lonE or latitude > latN or latitude < latS:
        continue
    point_fcst = get_point_fcst(fcst, longitude, latitude)
    point_fcst = point_fcst.expand_dims("station")
    point_fcst = point_fcst.assign_coords({"station": [station]})
    print(station)
    point_list.append(point_fcst)
point_ens_fcst = xr.concat(point_list, dim="station")
point_ens_fcst.to_netcdf(f"{RESULT_PATH}graphcast.nc")